In [104]:
%pylab
from scipy.integrate import cumtrapz
#from bokeh.models import Range1d
import matplotlib.pyplot as plt
import emcee
from multiprocessing import Pool
from scipy.optimize import minimize
from numba import njit
from scipy.stats import poisson

def inference_experiment(gamma=2.5, background=1e-1, aperture=30, N50=100,res=0.1):
    radii = generate_radii(gamma,background,aperture,N50,1.)
    rbins = np.linspace(0,aperture,int(aperture/res))
    N = np.histogram(radii,rbins)[0]
    p0 = (np.log10(2*(radii<1).sum()/(np.pi)),np.log10(N[-1]/np.diff(np.pi*rbins**2)[-1]), 0., gamma)
    #print(lossfunc(p0,rbins,N))
    #success = False
    sol = minimize(lossfunc, np.array(p0)+1e-4,args=(rbins,N),method='Nelder-Mead')
    if sol.success:    
        return sol.x[2:]# - np.array(p0)
    else:
        return [np.nan, np.nan]


def generate_radii(gamma=2.5, background=1e-1, Rmax=100, N50=100,a=1):
    r = np.logspace(-2,np.log10(Rmax),1000)
    norm = 2*np.pi/(gamma - 2)
    #cdf = 1 - (1 + r**2)**(1-gamma/2) 
    r50 = (2**(2/(gamma-2))-1)**0.5 
    sigma = norm * (1 + r**2)**(-gamma/2)
    ecdf = cumtrapz(2*np.pi*r*(sigma + background), r,initial=0);
    ecdf /= ecdf.max()
    frac_within_r50 = np.interp(r50,r,ecdf)
    sample_size = round(N50/frac_within_r50)
    samples = np.interp(np.sort(np.random.rand(sample_size)), ecdf, r)
    return samples*a

def lossfunc(x,rbins,bincounts):
    logmu0, logbackground, loga, gam = x
    mu, bg, a = 10**logmu0, 10**logbackground, 10**loga
    cumcounts_avg = 2*a**gam*np.pi*(a**(2-gam) - (a**2 + rbins**2)**(1-gam/2))* mu  / (gam-2) + np.pi*rbins**2*bg
    expected_counts = np.diff(cumcounts_avg)
    prob = poisson.logpmf(bincounts, expected_counts).sum()
    return -prob

#def fit_profile

Using matplotlib backend: TkAgg
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/mike/intel/oneapi/intelpython/latest/lib/python3.9/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['poisson']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [96]:
inference_experiment()

2.6645595769200794

In [105]:
from multiprocessing import Pool
#for i in range(1000): generate_radii()
default_params = [2.5,1e-1,30,10**2,0.1]
#result = [inference_experiment(*default_params) for i in range(1000)]
with Pool(15) as pool:
    result = np.array(pool.starmap(inference_experiment, 10000*[default_params]))
#results = []
#for i in range(100): results.append(inference_experiment()) #, range(10000))
#inference_experiment()

/tmp/ipykernel_125568/3410372765.py:15: RuntimeWarning: divide by zero encountered in log10
  p0 = (np.log10(2*(radii<1).sum()/(np.pi)),np.log10(N[-1]/np.diff(np.pi*rbins**2)[-1]), 0., gamma)
/home/mike/intel/oneapi/intelpython/latest/lib/python3.9/site-packages/scipy/optimize/_optimize.py:862: RuntimeWarning: invalid value encountered in subtract
  if (np.max(np.ravel(np.abs(sim[1:] - sim[0]))) <= xatol and
/home/mike/intel/oneapi/intelpython/latest/lib/python3.9/site-packages/scipy/optimize/_optimize.py:867: RuntimeWarning: invalid value encountered in subtract
  xr = (1 + rho) * xbar - rho * sim[-1]
/tmp/ipykernel_125568/3410372765.py:15: RuntimeWarning: divide by zero encountered in log10
  p0 = (np.log10(2*(radii<1).sum()/(np.pi)),np.log10(N[-1]/np.diff(np.pi*rbins**2)[-1]), 0., gamma)
/home/mike/intel/oneapi/intelpython/latest/lib/python3.9/site-packages/scipy/optimize/_optimize.py:862: RuntimeWarning: invalid value encountered in subtract
  if (np.max(np.ravel(np.abs(sim[1:] - s

In [120]:
#%ma
idx = (~np.any(np.isnan(result),axis=1)) * (result[:,1]>2)
result = result[idx]
plt.hist(10**result[:,0]*np.sqrt(2**(2/(np.array(result[:,1])-2))-1)/(2**(2/(2.5-2))-1)**0.5,np.linspace(0,10,100))
np.mean(np.log10(10**result[:,0]*np.sqrt(2**(2/(np.array(result[:,1])-2))-1)/(2**(2/(2.5-2))-1)**0.5))

0.15762468182812053

In [114]:
#np.max(10**result[:,0]*np.sqrt(2**(2/(np.array(result[:,1])-2))-1)/(2**(2/(2.5-2))-1)**0.5)
idx.sum()

8847

In [117]:
len(result)

8847

In [49]:
%lprun -f lossfunc inference_experiment()

2093.074018083576
2.5 2.952574685026501


Timer unit: 1e-09 s

Total time: 0.0451768 s
File: /tmp/ipykernel_24507/3469436785.py
Function: lossfunc at line 34

Line #      Hits         Time  Per Hit   % Time  Line Contents
    34                                           def lossfunc(x,rbins,bincounts):
    35       272     694987.0   2555.1      1.5      logmu0, logbackground, loga, gam = x
    36       272     421873.0   1551.0      0.9      mu, bg, a = 10**logmu0, 10**logbackground, 10**loga
    37       272   10836445.0  39839.9     24.0      cumcounts_avg = 2*a**gam*np.pi*(a**(2-gam) - (a**2 + rbins**2)**(1-gam/2))* mu  / (gam-2) + np.pi*rbins**2*bg
    38       272    2744743.0  10091.0      6.1      expected_counts = np.diff(cumcounts_avg)
    39       272   30379007.0 111687.5     67.2      prob = poisson.logpmf(bincounts, expected_counts).sum()
    40       272      99762.0    366.8      0.2      return -prob